In [ ]:
from __future__ import print_function
from __future__ import division

%matplotlib inline
import pandas as pd
import numpy as np

from matplotlib import pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
import statsmodels.api as sm

from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import (RBF,Matern, ConstantKernel,WhiteKernel,RationalQuadratic,Exponentiation,ExpSineSquared,
                                              DotProduct,
                                              ConstantKernel )

In [ ]:
# load test data
test_features = pd.read_csv('./dengue_features_test.csv',index_col=[0,1,2])


# load the provided data
train_features = pd.read_csv('./dengue_features_train.csv',
                             index_col=[0,1,2])

train_labels = pd.read_csv('./dengue_labels_train.csv',
                           index_col=[0,1,2])



# features used for approach 1 
features = [ 'ndvi_sw','reanalysis_dew_point_temp_k', 'reanalysis_specific_humidity_g_per_kg', 'reanalysis_min_air_temp_k',
               'station_min_temp_c',  'station_max_temp_c']

# features used for approach 2
features2 = ['reanalysis_min_air_temp_k','station_min_temp_c','reanalysis_dew_point_temp_k']


train_features = train_features[features]
test_features = test_features[features]

# Data Preprocessing

In [ ]:
# Data processing for both methods, need to rerun this part when start a new method

# Seperate data for San Juan
sj_train_features = train_features.loc['sj']
sj_test_features = test_features.loc['sj']
sj_train_labels = train_labels.loc['sj']

# Separate data for Iquitos
iq_train_features = train_features.loc['iq']
iq_test_features = test_features.loc['iq']
iq_train_labels = train_labels.loc['iq']

# fill nan with most recent value  could research and improve
sj_train_features.fillna(method='ffill', inplace=True)
sj_test_features.fillna(method='ffill', inplace=True)

iq_train_features.fillna(method='ffill', inplace=True)
iq_test_features.fillna(method='ffill', inplace=True)


sj_train_features = sj_train_features.reset_index('weekofyear')
sj_train_features = sj_train_features.reset_index('year')
sj_train_labels = sj_train_labels.reset_index('weekofyear')
sj_train_labels = sj_train_labels.reset_index('year')

sj_test_features = sj_test_features.reset_index('weekofyear')
sj_test_features = sj_test_features.reset_index('year')

iq_train_features = iq_train_features.reset_index('weekofyear')
iq_train_features = iq_train_features.reset_index('year')
iq_train_labels = iq_train_labels.reset_index('weekofyear')
iq_train_labels = iq_train_labels.reset_index('year')

iq_test_features = iq_test_features.reset_index('weekofyear')
iq_test_features = iq_test_features.reset_index('year')


df = sj_train_features
sj_val = df[656:]
sj_train = df[:656]
x_train = np.asarray(sj_train)
x_val = np.asarray(sj_val)
y_train = sj_train_labels['total_cases'][:656]
y_val = sj_train_labels['total_cases'][656:]

# log transform of cases
#y_train = [np.log(i) if i > 0 else 0 for i in y_train]
#y_val = [np.log(i) if i > 0 else 0 for i in y_val]

x_total = np.concatenate((x_train,x_val),axis=0)

y_total = np.asarray(sj_train_labels['total_cases'])


# Approach 1 

In [ ]:
#  function to create environmental lags for approach 1 
#  lag 1 specifies the farthest lag, lag2 specifies the closest lag
def create_envlags(a,lag1,lag2,endpoints=False):
    array = []
    for i in range(lagmax,len(a)): 
        temp = a[i-lag1:i-lag2+1,2:]
        temp = temp.flatten()

        array.append(temp)
    return np.asarray(array)

## San Juan Train Predict and Plot

In [ ]:
# feature preparation
lagmax=5
lagmin=3
x_val = np.concatenate((x_train[-lagmax:],x_val))
x_train = create_envlags(x_train,lagmax,lagmin)
x_val = create_envlags(x_val,lagmax,lagmin)
x_total = np.concatenate((x_train,x_val),axis=0)
y_train= y_train[lagmax:]
y_val = y_val

In [ ]:
# training and plotting result for San Juan 
len1 = 655-lagmax
len2 = 935-lagmax

k3 = 0.5**2* RationalQuadratic(length_scale=1, length_scale_bounds=(1e-2, 30.0), alpha=10,alpha_bounds=(1e-1,100))
#k4 = 1.* RBF(length_scale=[1.0 for i in range(24)], length_scale_bounds=(1e-2, 30)) + 1.**2 *ConstantKernel(0.1, (0.1, 10))*WhiteKernel(noise_level=1, noise_level_bounds=(1e-10, 1e10))
k5 = ConstantKernel(0.1, (0.01, 10.0))*(DotProduct(sigma_0=1.0, sigma_0_bounds=(0.1, 10.0)))
k4 = ConstantKernel(0.1, (0.1, 30))*1* Matern(length_scale=1, length_scale_bounds=(1e-1, 30),nu=1.5)+ ConstantKernel(0.1, (0.1, 20))*WhiteKernel(noise_level=0.1**2,
                  noise_level_bounds=(1e-3, 1e3))

#kernel = k3*k5+ k4 
kernel = k3*k5+k4
gp3 = GaussianProcessRegressor(kernel=kernel,random_state=1)
gp3.fit(x_train,y_train)
print(gp3.kernel_)

# Predict for the whole dataset (training and test)
y_pred3, sigma = gp3.predict(x_total, return_std=True)
fig = plt.figure(figsize=(20,10))
plt.plot(range(len1), y_pred3[:len1], 'b-', label=u'Prediction')
plt.plot(range(len1,len2), y_pred3[len1:len2],'r-',label=u'validation Prediction')
plt.plot(range(len2+1), y_total[lagmax:], label='totoalcases')


plt.xlabel('$x$')
plt.ylabel('$f(x)$')
plt.legend(loc='upper left')
plt.show()

In [ ]:
# MAE score evaluation
from sklearn.metrics import mean_absolute_error
print(mean_absolute_error(y_pred3[len1:len2],y_val[:280]))
print(mean_absolute_error(y_pred3[:len1+1],y_train))

## Iquitos train predict and plot

In [ ]:
# Feature preparation for Iquitos
split = 364
df = iq_train_features
iq_val = df[split:]
iq_train = df[:split]
x_train = np.asarray(iq_train)
x_val = np.asarray(iq_val)
y_train = iq_train_labels['total_cases'][:split]
y_val = iq_train_labels['total_cases'][split:]
x_total = np.concatenate((x_train,x_val),axis=0)
y_total = np.asarray(iq_train_labels['total_cases'])

lagmax=5
lagmin=3
x_val = np.concatenate((x_train[-lagmax:],x_val))
x_train = create_envlags(x_train,lagmax,lagmin)
x_val = create_envlags(x_val,lagmax,lagmin)
x_total = np.concatenate((x_train,x_val),axis=0)
y_train= y_train[lagmax:]
y_val = y_val

In [ ]:
# training and plotting for iquitos
k3 = 1* RationalQuadratic(length_scale=1, length_scale_bounds=(1e-1, 30.0), alpha=10,alpha_bounds=(1e-1,100))
k4 = 1.* RBF(length_scale=1, length_scale_bounds=(1e-2, 10)) + 1.**2 *ConstantKernel(0.1, (0.1, 10))*WhiteKernel(noise_level=1, noise_level_bounds=(1e-10, 1e10))
k5 = ConstantKernel(0.1, (0.01, 10.0))*(DotProduct(sigma_0=1.0, sigma_0_bounds=(0.1, 10.0)))


kernel = k3+k5*k4

# *ConstantKernel(0.1, (0.1, 10))
gp = GaussianProcessRegressor(kernel=kernel,random_state=1)
gp.fit(x_train,y_train)
print(gp.kernel_)
en1 = 364-lagmax
len2 = 520-lagmax

y_pred, sigma = gp.predict(x_total, return_std=True)
fig = plt.figure(figsize=(20,10))
plt.plot(range(len1), y_pred[:len1], 'b-', label=u'Prediction')
plt.plot(range(len1,len2), y_pred[len1:len2],'r-',label=u'validation Prediction')
plt.plot(range(len2), y_total[lagmax:], label='totoalcases')


plt.xlabel('$x$')
plt.ylabel('$f(x)$')
plt.legend(loc='upper left')
plt.show()

# Approach 2 

In [ ]:
# function to create lag features
def prepare_features(a,b,lag1,lag2,lag_case,include_week,include_year):
    array = []
    offset = 2
    if include_week:
        offset = 1
    if include_year:
        offset = 0
    for i in range(lagmax,len(a)): 
        
        temp = a[i-lag1:i-lag2+1,offset:]
        temp = temp.flatten()
        temp1 = b[i-lag_case:i]
       
        temp2 = np.concatenate((a[i,:2],temp))
        temp3 = np.concatenate((temp2,temp1))
        
        array.append(temp3)
    return np.asarray(array)

In [ ]:
# function used in recursive prediction
def create_one_feature(a,b,i,lagmax,lagmin,lagcase,include_week,include_year):
    offset = 2 
    if include_week:
        offset = 1
    if include_year:
        offset = 0
    temp = a[i-lagmax:i-lagmin+1,offset:]
    temp = temp.flatten()
    temp1 = b[i-lagcase:i]
    temp2 = np.concatenate((a[i,:2],temp))
    temp3 = np.concatenate((temp2,temp1))
    
    return temp3

## San juan train predict and plot

In [ ]:
# Feature preparation for San juan
lagmax=5
lagmin=3
lagcase=3
include_year = True
include_week = False
x_train_l = prepare_features(x_train,y_train,lagmax,lagmin,lagcase,include_week,include_year)
y_train_l= np.asarray(y_train[lagmax:])

In [ ]:
# Training
k1 = 1. *ConstantKernel(0.1, (0.1, 10))*RationalQuadratic(length_scale=1,length_scale_bounds=(1e-2, 20.0), alpha=10,alpha_bounds=(1e-2,100))

k3 = 1. * RBF(length_scale=[1.0 for i in range(20)] ,length_scale_bounds=(1e-1, 20.0))+1*ConstantKernel(0.1, (0.1, 10))*WhiteKernel(noise_level=1, noise_level_bounds=(1e-10, 1e10))


k4 = ConstantKernel(0.1, (0.01, 10.0))*(DotProduct(sigma_0=1.0, sigma_0_bounds=(0.1, 10.0)))

#k5 = ConstantKernel(0.1, (0.01, 10.0))*(DotProduct(sigma_0=1.0, sigma_0_bounds=(0.1, 10.0)))
kernel = k1*k4+k3
                                                                                                                   
gp_t = GaussianProcessRegressor(kernel=kernel,random_state=1)
gp_t.fit(x_train_l,y_train_l)
print(gp_t.kernel_)

In [ ]:
# Prediction
x_val_l = x_total[656-lagmax:936]
y_val_pred = np.asarray(y_train[656-lagmax:])
vallength = 280+lagmax

val_sigma = []
# recursive prediction
for i in range(lagmax,vallength):
    feature_val = create_one_feature(x_val_l,y_val_pred,i,lagmax,lagmin,lagcase,include_week,include_year)
    #print(feature_val.shape)
    predicted,sigma = gp_t.predict(feature_val.reshape(1,-1),return_std=True)
    y_val_pred = np.concatenate((y_val_pred,predicted))
    val_sigma.append(sigma)
    
y_val_pred = y_val_pred[lagmax:]
val_sigma = np.asarray(val_sigma).flatten()

y_train_pred,sigma_train = gp_t.predict(x_train_l,return_std=True)

y_pred_total = np.concatenate((y_train_pred,y_val_pred))
sigma = np.concatenate((sigma_train,val_sigma))

In [ ]:
# Plotting
fig = plt.figure(figsize=(20,10))

train_val_split = 655
total_length = 936


plt.plot(range(train_val_split-lagmax+1), y_train_pred, 'b-', label=u'Prediction')
plt.plot(range(train_val_split-lagmax+1,total_length-lagmax), y_val_pred,'r-',label=u'validation Prediction')
plt.plot(range(total_length-lagmax), y_total[lagmax:], label='totoalcases')

plt.xlabel('$x$')
plt.ylabel('$f(x)$')
plt.legend(loc='upper left')
plt.show()

## Iquitos Train Predict and Plot

In [ ]:
split = 364
df = iq_train_features
iq_val = df[split:]
iq_train = df[:split]
x_train = np.asarray(iq_train)
x_val = np.asarray(iq_val)
y_train = iq_train_labels['total_cases'][:split]
y_val = iq_train_labels['total_cases'][split:]
x_total = np.concatenate((x_train,x_val),axis=0)
y_total = np.asarray(iq_train_labels['total_cases'])

x_train_l = prepare_features(x_train,y_train,lagmax,lagmin,lagcase,include_week,include_year)
y_train_l= np.asarray(y_train[lagmax:])

In [ ]:
# Training
k1 = 1.**2 *ConstantKernel(0.1, (0.1, 10))*RationalQuadratic(length_scale=1,length_scale_bounds=(1e-2, 20.0), alpha=10,alpha_bounds=(1e-2,100))

k3 = 1. * RBF(length_scale=[1.0 for i in range(16)] ,length_scale_bounds=(1e-1, 20.0))+1.**2 *ConstantKernel(0.1, (0.1, 10))*WhiteKernel(noise_level=1, noise_level_bounds=(1e-10, 1e10))

k4 = ConstantKernel(0.1, (0.01, 10.0))*(DotProduct(sigma_0=1.0, sigma_0_bounds=(0.1, 10.0)))

kernel = k3+k1*k4

gp = GaussianProcessRegressor(kernel=kernel,random_state=1)
gp.fit(x_train_l,y_train_l)
print(gp.kernel_)

In [ ]:
# Prediction
length= 520
x_val_l = x_total[split-lagmax:520]
y_val_pred = np.asarray(y_train[split-lagmax:])
vallength = (length-split)+lagmax

# recursive prediction
for i in range(lagmax,vallength):
    feature_val = create_one_feature(x_val_l,y_val_pred,i,lagmax,lagmin,lagcase,include_week,include_year)
    #print(feature_val.shape)
    predicted = gp.predict(feature_val.reshape(1,-1))
    y_val_pred = np.concatenate((y_val_pred,predicted))

y_val_pred = y_val_pred[lagmax:]
y_train_pred = gp.predict(x_train_l)
y_pred_total = np.concatenate((y_train_pred,y_val_pred))

In [ ]:
# Plotting

fig = plt.figure(figsize=(20,10))


plt.plot(range(split-lagmax), y_train_pred, 'b-', label=u'Prediction')
plt.plot(range(split-lagmax,length-lagmax), y_val_pred,'r-',label=u'validation Prediction')
plt.plot(range(length-lagmax), y_total[lagmax:], label='totoalcases')



plt.xlabel('$x$')
plt.ylabel('$f(x)$')
plt.legend(loc='upper left')
plt.show()